# Title of Your Group Project

This Python notebook serves as a template for your group project for the course "Modeling in Cognitive Science".

This is the practical part of the group project where you get to implement the computational modeling workflow. In this part, you are expected to:


*   Implement at least two computational models relevant for your hypothesis. *(3 points)*
*   Simulate behavior from the two models. *(3 points)*
*   Implement a procedure for fitting the models to data. *(4 points)*
*   Implement a procedure for parameter recovery. *(5 points)*
*   (Implement a procedure for model recovery.) *(optional; 2 bonus points)*
*   Implement a model comparison. *(5 points)*.

You can gain a total of 20 points for the practical part of the group project.

**Note:** *Some of the exercises below (e.g. Model Simulation) rely on code from previous exercises (e.g., Model Implementation). In such cases, you are encouraged to rely on functions implemented for previous exercises. That is, you don't have to produce redundant code.*



## Model Implementation *(3 points)*

For this exercise you should:

*   Implement and simulate data from two* models that are suitable to test your hypothesis. *(3 points)*

<font size=2>*You may implement more than two models if you wish. However, two models are sufficient for this group project.</font>

Make sure to comment your code and provide an explanation for each code block in a preceding text block.


In [5]:
import numpy as np
# YOUR MODEL IMPLEMENTATION CODE GOES HERE

# environment of the experiment
class TowStepEnv:
    # constructor
    def __init__(self):
        self.state = 0
        self.action_space = [0, 1]
        self.state_space = [0, 1, 2]
        self.transition_prob = 0.7
        self.reward = 1
        self.terminal = False
        self.info = {}
        
        # matrix of transition probabilities
        # 0(action left) -> [0(stay in 0), p(go to 1), 1-p(go to 2)]
        # 1(action right) -> [0(stay in 0), 1-p(go to 1), p(go to 2)]
        self.stage_1_transition_matrix = np.array([[0, self.transition_prob, 1 - self.transition_prob], # action left
                                           [0, 1 - self.transition_prob, self.transition_prob]]) # action right
        
        self.seed = 0
        np.random.seed(self.seed)
        self.min_reward_prob = 0.25
        self.max_reward_prob = 0.75
        # matrix of reward probabilities
        # 0(state 0) -> [0 (left), 0(right)]
        # 1(state 1) -> [p1 (left), p2(right)]
        # 2(state 2) -> [p3 (left), p4(right)]
        p1 = np.random.uniform(self.min_reward_prob, self.max_reward_prob) 
        p2 = np.random.uniform(self.min_reward_prob, self.max_reward_prob)
        p3 = np.random.uniform(self.min_reward_prob, self.max_reward_prob)
        p4 = np.random.uniform(self.min_reward_prob, self.max_reward_prob)
        self.reward_prob_matrix = np.array([[0, 0], # first stage (state 0) for both actions
                                            [p1, p2], # second stage (state 1) for both actions
                                            [p3, p4]]) # second stage (state 2) for both actions
        
    
    def reset(self):
        self.state = 0
        self.terminal = False
        self.info = {}
        return self.state

    def step(self, action):
        if self.terminal:
            raise ValueError("Episode has already terminated")
        if action not in self.action_space:
            raise ValueError(f"The action: {action} is not valid, action space: {self.action_space}")

        # if in stage 1
        if self.state == 0:
            reward = self.reward_function(self.state, action) # reward will be 0
            self.state = np.random.choice(self.state_space, p=self.stage_1_transition_matrix[action])

            self.info["common_transition"] = self.is_common_state(self.state, action)
            self.info["state_transition_to"] = self.state
            self.info["reward_stage_1"] = reward
            self.info["action_stage_1"] = action
            # self.info["reward_probabilities_stage_1"] = self.reward_prob_matrix.flatten()
        
        # if in stage 2
        elif self.state in [1,2]:
            reward = self.reward_function(self.state, action)
            self.terminal = True
            self.info["reward_stage_2"] = reward
            self.info["action_stage_2"] = action
            # self.info["reward_probabilities"] = self.reward_prob_matrix.flatten()
            self.info["reward_probabilities"] = self.reward_prob_matrix.flatten()

        
        else:
            raise ValueError(f"state:{self.state} is an invalid state, state space: {self.state_space}")
        
        
        return self.state, reward, self.terminal, self.info
    
    # @classmethod
    def reward_function(self, state, action):
        if action not in self.action_space:
            raise ValueError(f"The action: {action} is not valid, action space: {self.action_space}")
        if state not in self.state_space:
            raise ValueError(f"state:{state} is an invalid state, state space: {self.state_space}")
        
        # give a reward according to the probability of getting a reward
        # for the action taken in the state ( state-action pair )
        reward = np.random.uniform() < self.reward_prob_matrix[state][action]
        # scale the reward for a costume reward value equal to self.reward
        # makes no difference in case self.reward = 1
        reward = reward * self.reward
        return reward
    
    def state_transition_function(self, state, action):
        if action not in self.action_space:
            raise ValueError(f"The action: {action} is not valid, action space: {self.action_space}")
        
        new_state = None
        terminal = False
        if state == 0:
            new_state = np.random.choice(self.state_space, p=self.stage_1_transition_matrix[action])
        elif state in [1,2]:
            terminal = True
        else:
            raise ValueError(f"state:{state} is an invalid state, state space: {self.state_space}")
        
        return new_state, terminal

    def is_common_state(self, state, action):
        if action not in self.action_space:
            raise ValueError(f"The action: {action} is not valid, action space: {self.action_space}")
        if state not in self.state_space:
            raise ValueError(f"state:{state} is an invalid state, state space: {self.state_space}")
        
        # return self.stage_1_transition_matrix[action, state] >= 0.5
        return self.stage_1_transition_matrix[action, state] == np.max(self.stage_1_transition_matrix[action])
        
# agent / models
# one agent who can have different evaluation algos / policies / models?

In [6]:
import pandas as pd
from IPython.display import display
from datetime import datetime

# simulate data 
# (for now from randome agent, as test the environment and task implementation)
def simulate_tow_step_task(env:TowStepEnv, agent=None, trails=200):
    env.reset()
    task_data = {}

    sd_for_random_walk = 0.025
    time_step = 0
    while time_step < trails:
        # first stage choice
        terminal = False
        while not terminal:
            action = np.random.choice([0,1]) # randome agent
            s, r, terminal, info = env.step(action)
        task_data[time_step] = info
        env.reset()
        # randome walk for the reward probabilies ( only for the 2 terminal states )
        env.reward_prob_matrix[1:] = random_walk_gaussian(env.reward_prob_matrix[1:], sd_for_random_walk)
        time_step += 1

    return task_data

def random_walk_gaussian(prob, sd):
    new_prob = prob + np.random.normal(scale = sd, size=np.shape(prob))
    return new_prob



task_data = simulate_tow_step_task(env=TowStepEnv(), trails=10)

task_df = pd.DataFrame.from_dict(task_data, orient='index')
task_df['trail_index'] = task_df.index

display(task_df)

time_identifier = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
task_df.to_csv(f"task_data_{time_identifier}.csv", index=False)

,common_transition,state_transition_to,reward_stage_1,action_stage_1,reward_stage_2,action_stage_2,reward_probabilities,trail_index
0,True,2,0,1,0,1,"[0.0, 0.0, 0.5244067519636624, 0.6075946831862...",0
1,False,2,0,0,1,0,"[0.0, 0.0, 0.5218262806688234, 0.6178596457346...",1
2,True,1,0,0,0,1,"[0.0, 0.0, 0.532922861487459, 0.62620150391902...",2
3,False,1,0,1,0,0,"[0.0, 0.0, 0.46909811609160706, 0.642541968805...",3
4,False,2,0,0,1,1,"[0.0, 0.0, 0.5074175964505685, 0.6792759380525...",4
5,True,1,0,0,1,0,"[0.0, 0.0, 0.5381748634687615, 0.7093354342721...",5
6,True,2,0,1,1,1,"[0.0, 0.0, 0.5254335589249702, 0.6983835767318...",6
7,True,2,0,1,1,0,"[0.0, 0.0, 0.5030468948951283, 0.7080561391783...",7
8,True,2,0,1,1,1,"[0.0, 0.0, 0.5023423391866619, 0.7187644359416...",8
9,True,2,0,1,1,0,"[0.0, 0.0, 0.48553082799226316, 0.709775606903...",9


In [7]:
# simple one run test
# np.random.seed(0)
env = TowStepEnv()

terminal = False
while not terminal:
    action = np.random.choice([0,1])
    s, r, terminal, info = env.step(action)

print(env.info)

{'common_transition': True, 'state_transition_to': 2, 'reward_stage_1': 0, 'action_stage_1': 1, 'reward_stage_2': 0, 'action_stage_2': 1, 'reward_probabilities': array([0.        , 0.        , 0.52440675, 0.60759468, 0.55138169,
       0.52244159])}


## Model Simulation *(3 points)*

For this exercise you should:

*   Simulate data from both models for a single set of parameters. The simulation should mimic the experiment you are trying to model. *(2 points)*

*   Plot the simulated behavior of both models. *(1 point)*

Make sure to comment your code and provide an explanation for each code block in a preceding text block.


In [ ]:
# YOUR MODEL SIMULATION CODE GOES HERE

## Parameter Fitting *(4 points)*

For this exercise you should:

*   Set up a suitable parameter search space *(1 point)*

*   Implement a procedure to evaluate the fit of a model based on data *(2 points)*

*   Implement a procedure for searching the parameter space. *(1 point)*

Make sure to comment your code and provide an explanation for each code block in a preceding text block.



In [ ]:
# YOUR PARAMETER FITTING CODE GOES HERE

## Parameter Recovery *(5 points)*

For this exercise you should:

*   Set up a suitable space of parameters relevant for parameter recovery *(1 point)*

*   Use the functions above to generate behavior from a models, for a given set of (randomly sampled) parameters, and then fit the model to its generated data. Make sure to evaluate the parameter fit in a quantiative manner. *(3 points)*

*   Plot the parameter recovery results for both models. *(1 point)*

Make sure to comment your code and provide an explanation for each code block in a preceding text block.





In [ ]:
# YOUR PARAMETER RECOVERY CODE GOES HERE

## *Optional*: Model Recovery *(2 bonus points)*

In this bonus exercise, you may examine model reovery. The bonus points count towards your total group project points. That is, you may accumlate up to 22 points in the practical part of the group project.

Make sure to comment your code and provide an explanation for each code block in a preceding text block.





In [ ]:
# YOUR MODEL RECOVERY CODE GOES HERE

## Model Comparison *(5 points)*

For this exercise you should:

*   Load and (potentially) preprocess the experimental data. (1 point)

*   Fit the two models to the data.  *(1 point)*

*   Evaluate which model performs better, taking into account fit and model complexity. *(2 points)*

*   Plot the behavior of the winning model against the data. *(1 point)**

Make sure to comment your code and provide an explanation for each code block in a preceding text block.





In [ ]:
# YOUR MODEL COMPARISON CODE GOES HERE